In [15]:
import numpy as np
from numpy.linalg import norm
import pickle
import matplotlib.pyplot as plt
import copy
import math
import time
import scipy
from pathlib import Path
import scipy.io

import cvxpy as cp

%matplotlib inline

# Extrapolation from the past: SDP for worst case $\|F(x^{1})\|^2 - \|F(x^0)\|^2$

We consider the following method:
$$
\widetilde{x}^0 = x^0,\quad \widetilde{x}^{k+1} = x^k - \gamma F(\widetilde{x}^k),\quad x^{k+1} = x^k - \gamma F(\widetilde{x}^{k+1})
$$
Note that iterates $\widetilde{x}^k$ follow exactly Optimistic Gradient Method update rule:
$$
\widetilde{x}^{k+1} = \widetilde{x}^k - 2\gamma F(\widetilde{x}^k) + \gamma F(\widetilde{x}^{k-1}) \quad \text{with} \quad \widetilde{x}^1 = \widetilde{x}^0 - \gamma F(\widetilde{x}^0)
$$

### Define parameters and the matrices for the problem

In [16]:
# Lipschitz parameter and stepsize
L = 1.0
gamma = 1.0 / (4*L)

# Matrices for SDP
M0 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, -1, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 1]])

M1 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 1, -1.0/2, 0],
                   [0, -1.0/2, 0, 0],
                   [0, 0, 0, 0]])

M2 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, (L**2)*(gamma**2)-1, 1, 0],
                   [0, 1, -1, 0],
                   [0, 0, 0, 0]])

M3 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 0, 1.0/2, 0],
                   [0, 1.0/2, 0, -1.0/2],
                   [0, 0, -1.0/2, 0]])

M4 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, -1, 0, 1],
                   [0, 0, (L**2)*(gamma**2), 0],
                   [0, 1, 0, -1]])

M5 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 0, -1.0/2, 1.0/2],
                   [0, -1.0/2, 1.0, -1.0/2],
                   [0, 1.0/2, -1.0/2, 0]])

M6 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, (L**2)*(gamma**2), -(L**2)*(gamma**2), 0],
                   [0, -(L**2)*(gamma**2), (L**2)*(gamma**2)-1, 1],
                   [0, 0, 1, -1]])

M7 = 1.0*np.array([[0, 0.5, 0, 0], 
                   [0.5, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

M8 = 1.0*np.array([[L**2, 0, 0, 0], 
                   [0, -1, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

M9 = 1.0*np.array([[0, 0, 0.5, 0], 
                   [0, 0, -gamma/2, 0],
                   [0.5, -gamma/2, 0, 0],
                   [0, 0, 0, 0]])

M10 = 1.0*np.array([[L**2, -(L**2)*gamma, 0, 0], 
                   [-(L**2)*gamma, (L**2)*(gamma**2), 0, 0],
                   [0, 0, -1, 0],
                   [0, 0, 0, 0]])

M11 = 1.0*np.array([[0, 0, 0, 0.5], 
                   [0, 0, 0, 0],
                   [0, 0, 0, -gamma/2],
                   [0.5, 0, -gamma/2, 0]])

M12 = 1.0*np.array([[(L**2), 0, -(L**2)*gamma, 0], 
                   [0, 0, 0, 0],
                   [-(L**2)*gamma, 0, (L**2)*(gamma**2), 0],
                   [0, 0, 0, -1]])

M13 = 1.0*np.array([[1, 0, 0, 0], 
                   [0, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

### Define and solve SDP problem

In [17]:
%%time
G = cp.Variable((4,4), symmetric=True)

constraints = [G >> 0]
constraints += [cp.trace(M1 @ G) >= 0]
constraints += [cp.trace(M2 @ G) >= 0]
constraints += [cp.trace(M3 @ G) >= 0]
constraints += [cp.trace(M4 @ G) >= 0]
constraints += [cp.trace(M5 @ G) >= 0]
constraints += [cp.trace(M6 @ G) >= 0]
constraints += [cp.trace(M7 @ G) >= 0]
constraints += [cp.trace(M8 @ G) >= 0]
constraints += [cp.trace(M9 @ G) >= 0]
constraints += [cp.trace(M10 @ G) >= 0]
constraints += [cp.trace(M11 @ G) >= 0]
constraints += [cp.trace(M12 @ G) >= 0]
constraints += [cp.trace(M13 @ G) == 1]

prob = cp.Problem(cp.Maximize(cp.trace(M0 @ G)),
                  constraints)

prob.solve()

Wall time: 36 ms


-1.1110082932697107e-05

In [18]:
print("The optimal value is", prob.value)
print("G = ")
print(G.value)

The optimal value is -1.1110082932697107e-05
G = 
[[1.0000053  0.09827903 0.09827698 0.09829571]
 [0.09827903 0.30086963 0.30084578 0.30086077]
 [0.09827698 0.30084578 0.30082181 0.30083944]
 [0.09829571 0.30086077 0.30083944 0.30085852]]


### Print dual variables

In [19]:
print("Dual variable 1 : ", constraints[1].dual_value)
print("Dual variable 2 : ", constraints[2].dual_value)
print("Dual variable 3 : ", constraints[3].dual_value)
print("Dual variable 4 : ", constraints[4].dual_value)
print("Dual variable 5 : ", constraints[5].dual_value)
print("Dual variable 6 : ", constraints[6].dual_value)
print("Dual variable 7 : ", constraints[7].dual_value)
print("Dual variable 8 : ", constraints[8].dual_value)
print("Dual variable 9 : ", constraints[9].dual_value)
print("Dual variable 10: ", constraints[10].dual_value)
print("Dual variable 11: ", constraints[11].dual_value)
print("Dual variable 12: ", constraints[12].dual_value)
print("Dual variable 13: ", constraints[13].dual_value)

Dual variable 1 :  4.001109602667928e-05
Dual variable 2 :  0.0
Dual variable 3 :  2.0000650018484327
Dual variable 4 :  0.0
Dual variable 5 :  0.5859157618454286
Dual variable 6 :  1.331117064465093
Dual variable 7 :  0.0
Dual variable 8 :  0.0
Dual variable 9 :  0.0
Dual variable 10:  0.0
Dual variable 11:  0.0
Dual variable 12:  0.0
Dual variable 13:  -2.0038238481393826e-05


# Extrapolation from the past: SDP for worst case $\|F(x^{2})\|^2 - \|F(x^1)\|^2$

We consider the following method:
$$
\widetilde{x}^0 = x^0,\quad \widetilde{x}^{k+1} = x^k - \gamma F(\widetilde{x}^k),\quad x^{k+1} = x^k - \gamma F(\widetilde{x}^{k+1})
$$
Note that iterates $\widetilde{x}^k$ follow exactly Optimistic Gradient Method update rule:
$$
\widetilde{x}^{k+1} = \widetilde{x}^k - 2\gamma F(\widetilde{x}^k) + \gamma F(\widetilde{x}^{k-1}) \quad \text{with} \quad \widetilde{x}^1 = \widetilde{x}^0 - \gamma F(\widetilde{x}^0)
$$

### Define parameters and the matrices for the problem

In [41]:
# Lipschitz parameter and stepsize
L = 1.0
gamma = 1.0 / (8*L)

# Matrices for SDP
M0 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, -1, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 1]])

M1 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 1, -1.0/2, 0, 0, 0],
                   [0, -1.0/2, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M2 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, (L**2)*(gamma**2)-1, 1, 0, 0, 0],
                   [0, 1, -1, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M3 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 1.0/2, 0, 0, 0],
                   [0, 1.0/2, 0, -1.0/2, 0, 0],
                   [0, 0, -1.0/2, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M4 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, -1, 0, 1, 0, 0],
                   [0, 0, (L**2)*(gamma**2), 0, 0, 0],
                   [0, 1, 0, -1, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M5 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, -1.0/2, 1.0/2, 0, 0],
                   [0, -1.0/2, 1.0, -1.0/2, 0, 0],
                   [0, 1.0/2, -1.0/2, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M6 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, (L**2)*(gamma**2), -(L**2)*(gamma**2), 0, 0, 0],
                   [0, -(L**2)*(gamma**2), (L**2)*(gamma**2)-1, 1, 0, 0],
                   [0, 0, 1, -1, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M7 = 1.0*np.array([[0, 0.5, 0, 0, 0, 0], 
                   [0.5, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M8 = 1.0*np.array([[L**2, 0, 0, 0, 0, 0], 
                   [0, -1, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M9 = 1.0*np.array([[0, 0, 0.5, 0, 0, 0], 
                   [0, 0, -gamma/2, 0, 0, 0],
                   [0.5, -gamma/2, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M10 = 1.0*np.array([[L**2, -(L**2)*gamma, 0, 0, 0, 0], 
                   [-(L**2)*gamma, (L**2)*(gamma**2), 0, 0, 0, 0],
                   [0, 0, -1, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M11 = 1.0*np.array([[0, 0, 0, 0.5, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, -gamma/2, 0, 0],
                   [0.5, 0, -gamma/2, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M12 = 1.0*np.array([[(L**2), 0, -(L**2)*gamma, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [-(L**2)*gamma, 0, (L**2)*(gamma**2), 0, 0, 0],
                   [0, 0, 0, -1, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M13 = 1.0*np.array([[1, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M14 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 1, 0, 0, 0],
                   [0, 1, 0, 0, -1, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, -1, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M15 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, -1, 0, 0, 1, 0],
                   [0, 0, 4*(L**2)*(gamma**2), 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 1, 0, 0, -1, 0],
                   [0, 0, 0, 0, 0, 0]])

M16 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0.5, 0, 0.5, 0],
                   [0, 0.5, 0, 0, 0, -0.5],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0.5, 0, 0, 0, -0.5],
                   [0, 0, -0.5, 0, -0.5, 0]])

M17 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, -1, 0, 0, 0, 1],
                   [0, 0, (L**2)*(gamma**2), 0, (L**2)*(gamma**2), 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, (L**2)*(gamma**2), 0, (L**2)*(gamma**2), 0],
                   [0, 1, 0, 0, 0, -1]])

M18 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, -0.5, 0, 0.5, 0],
                   [0, -0.5, 2, 0, -1, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0.5, -1, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M19 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, (L**2)*(gamma**2), -(L**2)*(gamma**2), 0, 0, 0],
                   [0, -(L**2)*(gamma**2), 4*(L**2)*(gamma**2)-1, 0, 1, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 1, 0, -1, 0],
                   [0, 0, 0, 0, 0, 0]])

M20 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, -0.5, 0, 0, 0.5],
                   [0, -0.5, 1, 0, 0.5, -0.5],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0.5, 0, 0, -0.5],
                   [0, 0.5, -0.5, 0, -0.5, 0]])

M21 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, (L**2)*(gamma**2), -(L**2)*(gamma**2), 0, -(L**2)*(gamma**2), 0],
                   [0, -(L**2)*(gamma**2), (L**2)*(gamma**2)-1, 0, (L**2)*(gamma**2), 1],
                   [0, 0, 0, 0, 0, 0],
                   [0, -(L**2)*(gamma**2), (L**2)*(gamma**2), 0, (L**2)*(gamma**2), 0],
                   [0, 0, 1, 0, 0, -1]])

M22 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0.5, -0.5, 0],
                   [0, 0, 0.5, 0, 0, 0],
                   [0, 0, -0.5, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M23 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, (L**2)*(gamma**2), 0, 0, 0],
                   [0, 0, 0, -1, 1, 0],
                   [0, 0, 0, 1, -1, 0],
                   [0, 0, 0, 0, 0, 0]])

M24 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 1, 0],
                   [0, 0, 0, 1, 0, -1],
                   [0, 0, 0, 0, -1, 0]])

M25 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, -1, 0, 1],
                   [0, 0, 0, 0, (L**2)*(gamma**2), 0],
                   [0, 0, 0, 1, 0, -1]])

M26 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, -0.5, 0.5],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, -0.5, 0, 1, -0.5],
                   [0, 0, 0.5, 0, -0.5, 0]])

M27 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, (L**2)*(gamma**2), 0, -(L**2)*(gamma**2), 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, -(L**2)*(gamma**2), 0, (L**2)*(gamma**2)-1, 1],
                   [0, 0, 0, 0, 1, -1]])

M28 = 1.0*np.array([[0, 0, 0, 0, 0.5, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, -gamma, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0.5, 0, -gamma, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M29 = 1.0*np.array([[L**2, 0, -2*(L**2)*gamma, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [-2*(L**2)*gamma, 0, 4*(L**2)*(gamma**2), 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, -1, 0],
                   [0, 0, 0, 0, 0, 0]])

M30 = 1.0*np.array([[0, 0, 0, 0, 0, 0.5], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, -gamma/2],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, -gamma/2],
                   [0.5, 0, -gamma/2, 0, -gamma/2, 0]])

M31 = 1.0*np.array([[L**2, 0, -(L**2)*gamma, 0, -(L**2)*gamma, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [-(L**2)*gamma, 0, (L**2)*(gamma**2), 0, (L**2)*(gamma**2), 0],
                   [0, 0, 0, 0, 0, 0],
                   [-(L**2)*gamma, 0, (L**2)*(gamma**2), 0, (L**2)*(gamma**2), 0],
                   [0, 0, 0, 0, 0, -1]])

### Define and solve SDP problem

In [42]:
%%time
G = cp.Variable((6,6), symmetric=True)

constraints = [G >> 0]
constraints += [cp.trace(M1 @ G) >= 0]
constraints += [cp.trace(M2 @ G) >= 0]
constraints += [cp.trace(M3 @ G) >= 0]
constraints += [cp.trace(M4 @ G) >= 0]
constraints += [cp.trace(M5 @ G) >= 0]
constraints += [cp.trace(M6 @ G) >= 0]
constraints += [cp.trace(M7 @ G) >= 0]
constraints += [cp.trace(M8 @ G) >= 0]
constraints += [cp.trace(M9 @ G) >= 0]
constraints += [cp.trace(M10 @ G) >= 0]
constraints += [cp.trace(M11 @ G) >= 0]
constraints += [cp.trace(M12 @ G) >= 0]
constraints += [cp.trace(M13 @ G) == 1]
constraints += [cp.trace(M14 @ G) >= 0]
constraints += [cp.trace(M15 @ G) >= 0]
constraints += [cp.trace(M16 @ G) >= 0]
constraints += [cp.trace(M17 @ G) >= 0]
constraints += [cp.trace(M18 @ G) >= 0]
constraints += [cp.trace(M19 @ G) >= 0]
constraints += [cp.trace(M20 @ G) >= 0]
constraints += [cp.trace(M21 @ G) >= 0]
constraints += [cp.trace(M22 @ G) >= 0]
constraints += [cp.trace(M23 @ G) >= 0]
constraints += [cp.trace(M24 @ G) >= 0]
constraints += [cp.trace(M25 @ G) >= 0]
constraints += [cp.trace(M26 @ G) >= 0]
constraints += [cp.trace(M27 @ G) >= 0]
constraints += [cp.trace(M28 @ G) >= 0]
constraints += [cp.trace(M29 @ G) >= 0]
constraints += [cp.trace(M30 @ G) >= 0]
constraints += [cp.trace(M31 @ G) >= 0]

prob = cp.Problem(cp.Maximize(cp.trace(M0 @ G)),
                  constraints)

prob.solve()

Wall time: 72.9 ms


2.0216040505671717e-05

In [43]:
print("The optimal value is", prob.value)
print("G = ")
print(G.value)

The optimal value is 2.0216040505671717e-05
G = 
[[1.00001237 0.0818657  0.08063682 0.08067285 0.08066447 0.08049986]
 [0.0818657  0.29416154 0.28971551 0.2895625  0.28954188 0.28956377]
 [0.08063682 0.28971551 0.28989662 0.28966806 0.28967098 0.28968162]
 [0.08067285 0.2895625  0.28966806 0.28950019 0.28949268 0.28949929]
 [0.08066447 0.28954188 0.28967098 0.28949268 0.28948959 0.28950278]
 [0.08049986 0.28956377 0.28968162 0.28949929 0.28950278 0.2895204 ]]


### Print dual variables

In [44]:
print("Dual variable 1 : ", constraints[1].dual_value)
print("Dual variable 2 : ", constraints[2].dual_value)
print("Dual variable 3 : ", constraints[3].dual_value)
print("Dual variable 4 : ", constraints[4].dual_value)
print("Dual variable 5 : ", constraints[5].dual_value)
print("Dual variable 6 : ", constraints[6].dual_value)
print("Dual variable 7 : ", constraints[7].dual_value)
print("Dual variable 8 : ", constraints[8].dual_value)
print("Dual variable 9 : ", constraints[9].dual_value)
print("Dual variable 10: ", constraints[10].dual_value)
print("Dual variable 11: ", constraints[11].dual_value)
print("Dual variable 12: ", constraints[12].dual_value)
print("Dual variable 13: ", constraints[13].dual_value)
print("Dual variable 14: ", constraints[14].dual_value)
print("Dual variable 15: ", constraints[15].dual_value)
print("Dual variable 16: ", constraints[16].dual_value)
print("Dual variable 17: ", constraints[17].dual_value)
print("Dual variable 18: ", constraints[18].dual_value)
print("Dual variable 19: ", constraints[19].dual_value)
print("Dual variable 20: ", constraints[20].dual_value)
print("Dual variable 21: ", constraints[21].dual_value)
print("Dual variable 22: ", constraints[22].dual_value)
print("Dual variable 23: ", constraints[23].dual_value)
print("Dual variable 24: ", constraints[24].dual_value)
print("Dual variable 25: ", constraints[25].dual_value)
print("Dual variable 26: ", constraints[26].dual_value)
print("Dual variable 27: ", constraints[27].dual_value)
print("Dual variable 28: ", constraints[28].dual_value)
print("Dual variable 29: ", constraints[29].dual_value)
print("Dual variable 30: ", constraints[30].dual_value)
print("Dual variable 31: ", constraints[31].dual_value)

Dual variable 1 :  0.0
Dual variable 2 :  0.0006246766415662874
Dual variable 3 :  0.0
Dual variable 4 :  0.00014031066660891262
Dual variable 5 :  0.0
Dual variable 6 :  0.04946584439865067
Dual variable 7 :  0.0
Dual variable 8 :  0.0
Dual variable 9 :  0.0
Dual variable 10:  0.0
Dual variable 11:  0.0
Dual variable 12:  0.0
Dual variable 13:  1.5584598201178295e-05
Dual variable 14:  0.0
Dual variable 15:  0.0
Dual variable 16:  0.0
Dual variable 17:  0.0
Dual variable 18:  0.0
Dual variable 19:  0.0
Dual variable 20:  0.0
Dual variable 21:  0.0
Dual variable 22:  0.0
Dual variable 23:  0.0
Dual variable 24:  1.0000222226242979
Dual variable 25:  0.0
Dual variable 26:  0.051484087917466025
Dual variable 27:  1.1160970364753555
Dual variable 28:  0.0
Dual variable 29:  0.0
Dual variable 30:  0.0
Dual variable 31:  0.0


### Define and solve SDP problem with 4 constraints

In [48]:
# Lipschitz parameter and stepsize
L = 1.0
gamma = 1.0 / (8*L)

# Matrices for SDP
M0 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, -1, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 1]])

M1 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 1, -1.0/2, 0, 0, 0],
                   [0, -1.0/2, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M2 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, (L**2)*(gamma**2)-1, 1, 0, 0, 0],
                   [0, 1, -1, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M3 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 1.0/2, 0, 0, 0],
                   [0, 1.0/2, 0, -1.0/2, 0, 0],
                   [0, 0, -1.0/2, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M4 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, -1, 0, 1, 0, 0],
                   [0, 0, (L**2)*(gamma**2), 0, 0, 0],
                   [0, 1, 0, -1, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M5 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, -1.0/2, 1.0/2, 0, 0],
                   [0, -1.0/2, 1.0, -1.0/2, 0, 0],
                   [0, 1.0/2, -1.0/2, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M6 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, (L**2)*(gamma**2), -(L**2)*(gamma**2), 0, 0, 0],
                   [0, -(L**2)*(gamma**2), (L**2)*(gamma**2)-1, 1, 0, 0],
                   [0, 0, 1, -1, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M7 = 1.0*np.array([[0, 0.5, 0, 0, 0, 0], 
                   [0.5, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M8 = 1.0*np.array([[L**2, 0, 0, 0, 0, 0], 
                   [0, -1, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M9 = 1.0*np.array([[0, 0, 0.5, 0, 0, 0], 
                   [0, 0, -gamma/2, 0, 0, 0],
                   [0.5, -gamma/2, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M10 = 1.0*np.array([[L**2, -(L**2)*gamma, 0, 0, 0, 0], 
                   [-(L**2)*gamma, (L**2)*(gamma**2), 0, 0, 0, 0],
                   [0, 0, -1, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M11 = 1.0*np.array([[0, 0, 0, 0.5, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, -gamma/2, 0, 0],
                   [0.5, 0, -gamma/2, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M12 = 1.0*np.array([[(L**2), 0, -(L**2)*gamma, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [-(L**2)*gamma, 0, (L**2)*(gamma**2), 0, 0, 0],
                   [0, 0, 0, -1, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M13 = 1.0*np.array([[1, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M14 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 1, 0, 0, 0],
                   [0, 1, 0, 0, -1, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, -1, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M15 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, -1, 0, 0, 1, 0],
                   [0, 0, 4*(L**2)*(gamma**2), 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 1, 0, 0, -1, 0],
                   [0, 0, 0, 0, 0, 0]])

M16 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0.5, 0, 0.5, 0],
                   [0, 0.5, 0, 0, 0, -0.5],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0.5, 0, 0, 0, -0.5],
                   [0, 0, -0.5, 0, -0.5, 0]])

M17 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, -1, 0, 0, 0, 1],
                   [0, 0, (L**2)*(gamma**2), 0, (L**2)*(gamma**2), 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, (L**2)*(gamma**2), 0, (L**2)*(gamma**2), 0],
                   [0, 1, 0, 0, 0, -1]])

M18 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, -0.5, 0, 0.5, 0],
                   [0, -0.5, 2, 0, -1, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0.5, -1, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M19 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, (L**2)*(gamma**2), -(L**2)*(gamma**2), 0, 0, 0],
                   [0, -(L**2)*(gamma**2), 4*(L**2)*(gamma**2)-1, 0, 1, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 1, 0, -1, 0],
                   [0, 0, 0, 0, 0, 0]])

M20 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, -0.5, 0, 0, 0.5],
                   [0, -0.5, 1, 0, 0.5, -0.5],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0.5, 0, 0, -0.5],
                   [0, 0.5, -0.5, 0, -0.5, 0]])

M21 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, (L**2)*(gamma**2), -(L**2)*(gamma**2), 0, -(L**2)*(gamma**2), 0],
                   [0, -(L**2)*(gamma**2), (L**2)*(gamma**2)-1, 0, (L**2)*(gamma**2), 1],
                   [0, 0, 0, 0, 0, 0],
                   [0, -(L**2)*(gamma**2), (L**2)*(gamma**2), 0, (L**2)*(gamma**2), 0],
                   [0, 0, 1, 0, 0, -1]])

M22 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0.5, -0.5, 0],
                   [0, 0, 0.5, 0, 0, 0],
                   [0, 0, -0.5, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M23 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, (L**2)*(gamma**2), 0, 0, 0],
                   [0, 0, 0, -1, 1, 0],
                   [0, 0, 0, 1, -1, 0],
                   [0, 0, 0, 0, 0, 0]])

M24 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 1, 0],
                   [0, 0, 0, 1, 0, -1],
                   [0, 0, 0, 0, -1, 0]])

M25 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, -1, 0, 1],
                   [0, 0, 0, 0, (L**2)*(gamma**2), 0],
                   [0, 0, 0, 1, 0, -1]])

M26 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, -0.5, 0.5],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, -0.5, 0, 1, -0.5],
                   [0, 0, 0.5, 0, -0.5, 0]])

M27 = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, (L**2)*(gamma**2), 0, -(L**2)*(gamma**2), 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, -(L**2)*(gamma**2), 0, (L**2)*(gamma**2)-1, 1],
                   [0, 0, 0, 0, 1, -1]])

M28 = 1.0*np.array([[0, 0, 0, 0, 0.5, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, -gamma, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0.5, 0, -gamma, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0]])

M29 = 1.0*np.array([[L**2, 0, -2*(L**2)*gamma, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [-2*(L**2)*gamma, 0, 4*(L**2)*(gamma**2), 0, 0, 0],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, -1, 0],
                   [0, 0, 0, 0, 0, 0]])

M30 = 1.0*np.array([[0, 0, 0, 0, 0, 0.5], 
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, -gamma/2],
                   [0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, -gamma/2],
                   [0.5, 0, -gamma/2, 0, -gamma/2, 0]])

M31 = 1.0*np.array([[L**2, 0, -(L**2)*gamma, 0, -(L**2)*gamma, 0], 
                   [0, 0, 0, 0, 0, 0],
                   [-(L**2)*gamma, 0, (L**2)*(gamma**2), 0, (L**2)*(gamma**2), 0],
                   [0, 0, 0, 0, 0, 0],
                   [-(L**2)*gamma, 0, (L**2)*(gamma**2), 0, (L**2)*(gamma**2), 0],
                   [0, 0, 0, 0, 0, -1]])

In [49]:
%%time
G = cp.Variable((6,6), symmetric=True)

constraints = [G >> 0]
# constraints += [cp.trace(M1 @ G) >= 0]
constraints += [cp.trace(M2 @ G) >= 0]
# constraints += [cp.trace(M3 @ G) >= 0]
# constraints += [cp.trace(M4 @ G) >= 0]
# constraints += [cp.trace(M5 @ G) >= 0]
constraints += [cp.trace(M6 @ G) >= 0]
# constraints += [cp.trace(M7 @ G) >= 0]
# constraints += [cp.trace(M8 @ G) >= 0]
# constraints += [cp.trace(M9 @ G) >= 0]
# constraints += [cp.trace(M10 @ G) >= 0]
# constraints += [cp.trace(M11 @ G) >= 0]
# constraints += [cp.trace(M12 @ G) >= 0]
constraints += [cp.trace(M13 @ G) == 1]
# constraints += [cp.trace(M14 @ G) >= 0]
# constraints += [cp.trace(M15 @ G) >= 0]
# constraints += [cp.trace(M16 @ G) >= 0]
# constraints += [cp.trace(M17 @ G) >= 0]
# constraints += [cp.trace(M18 @ G) >= 0]
# constraints += [cp.trace(M19 @ G) >= 0]
# constraints += [cp.trace(M20 @ G) >= 0]
# constraints += [cp.trace(M21 @ G) >= 0]
# constraints += [cp.trace(M22 @ G) >= 0]
# constraints += [cp.trace(M23 @ G) >= 0]
constraints += [cp.trace(M24 @ G) >= 0]
# constraints += [cp.trace(M25 @ G) >= 0]
# constraints += [cp.trace(M26 @ G) >= 0]
constraints += [cp.trace(M27 @ G) >= 0]
# constraints += [cp.trace(M28 @ G) >= 0]
# constraints += [cp.trace(M29 @ G) >= 0]
# constraints += [cp.trace(M30 @ G) >= 0]
# constraints += [cp.trace(M31 @ G) >= 0]

prob = cp.Problem(cp.Maximize(cp.trace(M0 @ G)),
                  constraints)

prob.solve()

Wall time: 20 ms


8.344778729907354e-06

In [50]:
print("The optimal value is", prob.value)
print("G = ")
print(G.value)

The optimal value is 8.344778729907354e-06
G = 
[[0.99999462 0.         0.         0.         0.         0.        ]
 [0.         0.51687393 0.45251315 0.45240911 0.45241294 0.4524244 ]
 [0.         0.45251315 0.39621777 0.39608178 0.39609686 0.39611623]
 [0.         0.45240911 0.39608178 0.39601083 0.39600985 0.39601353]
 [0.         0.45241294 0.39609686 0.39600985 0.39600617 0.39601185]
 [0.         0.4524244  0.39611623 0.39601353 0.39601185 0.39601917]]


### Print dual variables

In [51]:
print("Dual variable 1 : ", constraints[1].dual_value)
print("Dual variable 2 : ", constraints[2].dual_value)
print("Dual variable 3 : ", constraints[3].dual_value)
print("Dual variable 4 : ", constraints[4].dual_value)
print("Dual variable 5 : ", constraints[5].dual_value)
# print("Dual variable 6 : ", constraints[6].dual_value)
# print("Dual variable 7 : ", constraints[7].dual_value)
# print("Dual variable 8 : ", constraints[8].dual_value)
# print("Dual variable 9 : ", constraints[9].dual_value)
# print("Dual variable 10: ", constraints[10].dual_value)
# print("Dual variable 11: ", constraints[11].dual_value)
# print("Dual variable 12: ", constraints[12].dual_value)
# print("Dual variable 13: ", constraints[13].dual_value)
# print("Dual variable 14: ", constraints[14].dual_value)
# print("Dual variable 15: ", constraints[15].dual_value)
# print("Dual variable 16: ", constraints[16].dual_value)
# print("Dual variable 17: ", constraints[17].dual_value)
# print("Dual variable 18: ", constraints[18].dual_value)
# print("Dual variable 19: ", constraints[19].dual_value)
# print("Dual variable 20: ", constraints[20].dual_value)
# print("Dual variable 21: ", constraints[21].dual_value)
# print("Dual variable 22: ", constraints[22].dual_value)
# print("Dual variable 23: ", constraints[23].dual_value)
# print("Dual variable 24: ", constraints[24].dual_value)
# print("Dual variable 25: ", constraints[25].dual_value)
# print("Dual variable 26: ", constraints[26].dual_value)
# print("Dual variable 27: ", constraints[27].dual_value)
# print("Dual variable 28: ", constraints[28].dual_value)
# print("Dual variable 29: ", constraints[29].dual_value)
# print("Dual variable 30: ", constraints[30].dual_value)
# print("Dual variable 31: ", constraints[31].dual_value)

Dual variable 1 :  0.0006056605420823466
Dual variable 2 :  0.03553547320564313
Dual variable 3 :  -1.300957328058842e-06
Dual variable 4 :  0.999991859070287
Dual variable 5 :  1.6247489566354394


In [52]:
G_val = G.value
M_aux = 1.0*np.array([[0, 0, 0, 0, 0, 0], 
                     [0, (L**4)*(gamma**4)*0.5, 0, 0, 0, 0],
                     [0, 0, 10.0*(L**2)*(gamma**2)/3, 0, -10.0*(L**2)*(gamma**2)/3, 0],
                     [0, 0, 0, 0, 0, 0],
                     [0, 0, -10.0*(L**2)*(gamma**2)/3, 0, 10.0*(L**2)*(gamma**2)/3, 0],
                     [0, 0, 0, 0, 0, 0]])

np.trace(np.dot(M_aux, G_val))

6.466900620568472e-05